# Embeddings Neuronales


**TODO**

### Importanción de librería requeridas

In [27]:
import gensim.corpora as corpora
from gensim.models import Word2Vec, FastText

from collections import Counter
import numpy as np

import pickle
import pandas as pd

from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline


### Definición de variables globales

In [66]:
TEXT_FILE_READ = 'docs/preprocessing_reddit_data.csv'
TEXT_SAVE_FILE = 'docs/reddit_data_fasttext.csv'
FILENAME_PICKLE = "docs/tmpreddit.pickle"
n_clusters = 70

### Lectura de los comentarios de Reddit

Los comentarios fueron previamente preprocesados (Ver en TODO).

In [29]:
with open(FILENAME_PICKLE, 'rb') as f:
    df = pickle.load(f)


### Vocabulario

In [45]:
# Create Dictionary
id2word = corpora.Dictionary(df['lemma_tokens'])

# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)

# Creating a corpus object
corpus = [id2word.doc2bow(d) for d in df['lemma_tokens']]

processed_corpus = df['lemma_tokens']


### Entrenamiento del modelo FastText

In [55]:
model = FastText(sentences=processed_corpus, vector_size=100, window=5, min_count=1, workers=4)
model.train(processed_corpus, total_examples=len(processed_corpus), epochs=100)

(18381159, 18695800)

In [ ]:
# algunas predicciones
model.get_nearest_neighbors('peron')

In [40]:
model.get_nearest_neighbors('cristina')

[(0.9955493211746216, 'cristi'),
 (0.9921181797981262, 'kirchnerista'),
 (0.9887705445289612, 'cristina_fernandez'),
 (0.9884024858474731, 'cristina_fernández'),
 (0.988014280796051, 'cristín'),
 (0.9856591820716858, 'kirchnerismo'),
 (0.9838628172874451, 'fernández'),
 (0.9806126952171326, 'argentino"'),
 (0.9791547060012817, 'turista'),
 (0.9788342118263245, 'ladrona')]

In [41]:
model.get_nearest_neighbors('nestor')

[(0.9962456822395325, 'cfk'),
 (0.9956827163696289, 'lopez'),
 (0.9956071376800537, 'seinfeld'),
 (0.9955170154571533, 'zombie'),
 (0.9954903721809387, 'remake'),
 (0.9953209161758423, 'bullrich'),
 (0.9948703646659851, 'stealth'),
 (0.9945695400238037, 'users'),
 (0.9945260286331177, 'stewart'),
 (0.9945029616355896, 'evil')]

In [34]:
model.get_nearest_neighbors('neoliberal')

[(0.9928846955299377, 'economy'),
 (0.9909766316413879, 'neoliberalismo'),
 (0.9904403686523438, 'nací'),
 (0.9893696904182434, 'republica_argentina'),
 (0.9881485104560852, 'ladron'),
 (0.9877192378044128, 'fernandez'),
 (0.9875622391700745, 'crea'),
 (0.986701488494873, 'arg'),
 (0.9864534735679626, 'referis'),
 (0.9864226579666138, 'cereal')]

In [35]:
model.get_nearest_neighbors('malvinas')

[(0.9982995390892029, 'doctrina'),
 (0.9981448650360107, 'petróleo'),
 (0.9979062676429749, 'turbina'),
 (0.9976403117179871, 'juez'),
 (0.9974097609519958, 'marihuán'),
 (0.9973348379135132, 'gris'),
 (0.9971646070480347, 'nena'),
 (0.9970555901527405, 'wanda_nara'),
 (0.9969688057899475, 'gorila'),
 (0.9969642758369446, 'película')]

In [38]:
model.get_analogies('cristina', 'ladrona', 'alberto')

[(0.9841022491455078, 'malvina'),
 (0.9838746190071106, 'títerir'),
 (0.9836748242378235, 'dona'),
 (0.9829391837120056, 'malvinas'),
 (0.9824694395065308, 'doctrina'),
 (0.9824132919311523, 'estatua'),
 (0.9818088412284851, 'reina'),
 (0.9814509153366089, 'infarto'),
 (0.9808228015899658, 'estafa'),
 (0.980640709400177, 'archivo')]

In [77]:
model.wv.most_similar('mapuche')

[('mapuchez', 0.975992739200592),
 ('mapuches', 0.9645689725875854),
 ('mapuchecoin', 0.9489633440971375),
 ('mapuch', 0.9372188448905945),
 ('no-mapuches', 0.9177072048187256),
 ('pseudomapuche', 0.9131770133972168),
 ('mapu', 0.8864879012107849),
 ('mapuchir', 0.8846395611763),
 ('pseudo-mapuches', 0.8769819736480713),
 ('pseudomapuch', 0.8387230038642883)]

### Generación de vectores desde documentos

In [57]:
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features
    
vectorized_docs = vectorize(processed_corpus, model=model)
len(vectorized_docs), len(vectorized_docs[0])



(27791, 100)

### Generación de clusters

In [58]:
def mbkmeans_clusters(
	X, 
    k, 
    mb, 
    print_silhouette_values, 
):
    """Generate clusters and print Silhouette metrics using MBKmeans

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [59]:
clustering, cluster_labels = mbkmeans_clusters(
	X=vectorized_docs,
    k=n_clusters,
    mb=500,
    print_silhouette_values=True,
)
df_clusters = pd.DataFrame({
    "text": df["body"].values,
    "tokens": [" ".join(text) for text in processed_corpus],
    "cluster": cluster_labels
})



For n_clusters = 70
Silhouette coefficient: 0.01
Inertia:1635970.9632375669
Silhouette values:
    Cluster 13: Size:223 | Avg:0.22 | Min:0.07 | Max: 0.38
    Cluster 49: Size:156 | Avg:0.14 | Min:-0.05 | Max: 0.31
    Cluster 67: Size:53 | Avg:0.11 | Min:-0.07 | Max: 0.34
    Cluster 11: Size:349 | Avg:0.11 | Min:-0.05 | Max: 0.27
    Cluster 28: Size:236 | Avg:0.09 | Min:-0.06 | Max: 0.26
    Cluster 21: Size:139 | Avg:0.09 | Min:-0.07 | Max: 0.27
    Cluster 42: Size:90 | Avg:0.09 | Min:-0.08 | Max: 0.30
    Cluster 26: Size:189 | Avg:0.08 | Min:-0.08 | Max: 0.29
    Cluster 63: Size:303 | Avg:0.08 | Min:-0.05 | Max: 0.26
    Cluster 27: Size:90 | Avg:0.07 | Min:-0.06 | Max: 0.26
    Cluster 45: Size:207 | Avg:0.07 | Min:-0.09 | Max: 0.26
    Cluster 23: Size:207 | Avg:0.07 | Min:-0.07 | Max: 0.28
    Cluster 34: Size:92 | Avg:0.07 | Min:-0.05 | Max: 0.23
    Cluster 33: Size:256 | Avg:0.06 | Min:-0.08 | Max: 0.24
    Cluster 38: Size:231 | Avg:0.05 | Min:-0.07 | Max: 0.26
    Cluste

### *Top terms* por cluster (basado en los centroides de los clusters)

In [69]:
print("Most representative terms per cluster (based on centroids):")
for i in range(n_clusters):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=10)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")



Most representative terms per cluster (based on centroids):
Cluster 0: macristo macri~~. macris macrium macri-peño macri mracri macrismo macristaaaa -macri 
Cluster 1: desgraciada desgracia felizmente \*gracia gracia desgraciado agradecida graciassss graciasss desagraciado 
Cluster 2: perpetúar perfumar permítanmar perimetro perpetuar permanente perturbadoras percatar perpetrado perturbador 
Cluster 3: viejo abejo cejo vieja.me.dejo viejo= pendeviejo viviente rejo festejo vieja 
Cluster 4: rindo festilindo lindo gondo brindo cundo hundo oriundo swgundo trotamundo 
Cluster 5: argentina?."al argentina-perú argentiniar argentino argentinatm argentina1234 argentinans aegentino argentinoo argentinadepie 
Cluster 6: cavar caducó cat6 caús caliente cararrotar carvajal caroyar capua caído 
Cluster 7: desmadrar enterrar desenterrar yikar oar depurar tetrar descifrar delirar ríar 
Cluster 8: agua calientenla caliente agua\ piojo aguja aguila freelo riachuelo pintamelo 
Cluster 9: pesos dólar pes

### *Top terms* por cluster (basado en las palabras más frecuentes)

In [61]:
for i in range(n_clusters):
    tokens_per_cluster = ""
    most_frequent = Counter(" ".join(df_clusters.query(f"cluster == {i}")["tokens"]).split()).most_common(5)
    for t in most_frequent:
        tokens_per_cluster += f"{t[0]}({str(t[1])}) "
    print(f"Cluster {i}: {tokens_per_cluster}")


Cluster 0: macri(76) alberto(54) /s(40) cristina(28) kirchner(18) 
Cluster 1: vida(167) gracias(139) op(33) gracia(32) dios(23) 
Cluster 2: perro(91) llamar(82) andar(27) pensar(23) mandar(23) 
Cluster 3: viejo(226) hijo(58) vivir(39) morir(34) puta(33) 
Cluster 4: lindo(110) mundo(84) che(32) mierda(31) mierdo(25) 
Cluster 5: argentino(230) argentina(153) mundo(16) r(15) pasar(13) 
Cluster 6: cara(107) casa(101) caer(84) cagar(63) calle(57) 
Cluster 7: dejar(212) pasar(60) entrar(57) tirar(49) querer(30) 
Cluster 8: agua(88) calor(81) orto(45) ojo(41) aire(39) 
Cluster 9: peso(117) dolar(110) pesos(72) dólar(64) dólares(33) 
Cluster 10: hablar(181) dejar(8) cabeza(7) escuchar(7) cristina(6) 
Cluster 11: vo(365) decir(30) él(28) so(16) pasar(15) 
Cluster 12: gente(82) vivir(75) provincia(56) nacional(44) querer(41) 
Cluster 13: the(195) of(74) and(70) you(63) to(60) 
Cluster 14: mano(100) man(6) querer(5) humano(4) duro(4) 
Cluster 15: matar(84) robar(33) contar(15) nisman(10) evitar(9

### Recupere los documentos más representativos (basados en los centroides de los clústeres) para un cluster en particular

In [70]:
test_cluster = 0
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:10]:
    print( df["body"].values[d])
    print("-------------")

No peronistas: Yrigoyen (1916 - 1922), Alvear y Macri.. Peronistas: Perón (1946 - 1952), Menem, Kirchner y CFK.. &#x200B;. Antes del 2000, la diferencia no era tan grande.
-------------
Macri=bad Cristina=good los pasacalles de mayra dicen vota la lista de Cristina y al bigote que tenemos de presidente lo limpiaron de todos los afiches.
-------------
Es una catarata infinita de falacias y sesgos, podría escribir un libro comentado lo objetable. Sigue siendo mejor que un talibán kirchnerista pero .. me hace acordar mucho a macri, kirchnerismo con buenos modales
-------------
La oposición conformada por Fernanda Vallejos, El Gato Silvestre, y liderado por la mismísima Cristina F Kirchner
-------------
No entiendo qué piensan cuando reciben noticias como lo de la plata enterrada de Máximo, los dólares traídos de contrabando para campaña de Antonini Wilson, la rosadita y todas esas cosas ya probadas ¿Solo ah, pero Macri?
-------------
Qué es lo q festejan? Por qué sonríen, cantan y alaban 

In [63]:

#print(len(vectorized_docs))
#print(vectorized_docs[0])

test_v = vectorize([['defender', 'peso', 'siente', 'corazón', 'compro', 'pesos', 'tasa', 'fijo', 'año']], model=model)
prediction = clustering.predict(test_v)
print(prediction)

[9]


In [64]:
reddit = pd.read_csv(TEXT_FILE_READ)

def get_cluster(row):
    test_v = vectorize([row], model=model)
    return clustering.predict(test_v)

reddit['cluster'] = reddit.apply(lambda row: get_cluster(row['lemma_tokens']) , axis = 1) 


In [65]:
# Show
reddit.head(10)

,score,id,flair,comms_num,body,comment_parent_id,is_replay,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,lemma_tokens,body_preprocessing,cluster
0,1,hfw14mt,Discusion🧐,1,todo para decir que tapaste el baño. tira un b...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['tapastir', 'baño', 'tirar', 'balde', 'aguo']",tapastir baño tirar balde aguo,[18]
1,1,hfw41eh,Discusion🧐,0,"sopapa primero master, si hay tapón te vas a t...",hfw14mt,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sopapa', 'master', 'tapón', 'va', 'teñir', '...",sopapa master tapón va teñir medio,[18]
2,1,hfw1ao2,Discusion🧐,0,"Usas la sopapa, o tiras agua caliente con un b...",q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sopapo', 'tira', 'agua', 'caliente', 'balde']",sopapo tira agua caliente balde,[18]
3,1,hfw3jof,Discusion🧐,2,Lo que he probado que siempre me dio resultado...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['probado', 'resultado', 'sellar', 'boca', 'in...",probado resultado sellar boca inodoro tirar ca...,[18]
4,1,hfw6v4i,Discusion🧐,0,Estas cobrando por dar mantenimiento y no sabe...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['cobrar', 'mantenimiento', 'carajo', 'kjjjjjj...",cobrar mantenimiento carajo kjjjjjjjjj vivirio...,[18]
5,1,hfw26iv,Discusion🧐,0,"Si tenes algo con punta, metelo y hace un poco...",q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['tén', 'punto', 'metelo', 'fuerza', 'romper',...",tén punto metelo fuerza romper tapo baño tirar...,[18]
6,1,hfw2gof,Discusion🧐,1,"Con una manguera para regar el jardín, si tene...",q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['regar', 'jardín', 'tén', 'pod', 'probar']",regar jardín tén pod probar,[18]
7,1,hfw5s13,Discusion🧐,0,"despues regas el jardin y se lava sola, solo q...",hfw2gof,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['rega', 'jardin', 'lava', 'tenés', 'lavarte',...",rega jardin lava tenés lavarte mano pulgar chorro,[18]
8,1,hfw3air,Discusion🧐,0,La respuesta real es que se venden unos caños ...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['respuesta', 'real', 'vender', 'caño', 'alamb...",respuesta real vender caño alambrado decir cañ...,[18]
9,7,hfvxa6w,Discusion🧐,3,Mi alfajor favorito es el Havana,q443eo,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['alfajor', 'favorito', 'haván']",alfajor favorito haván,[18]


In [67]:
reddit.to_csv(TEXT_SAVE_FILE, index=False)

In [68]:
for i in range(n_clusters):
    reddit[(reddit["cluster"] == i)][['flair', 'body']].to_csv('docs/testfasttext/' + str(i) + '.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'docs/testfasttext/0.csv'